![banner](https://raw.githubusercontent.com/priyammaz/HAL-DL-From-Scratch/main/src/visuals/banner.png)

# Transfer Learning and PreTrained Models

Before we get into building a ton of models, I want to first talk about one of the coolest aspects of Deep Learning: **Transfer Learning**. As we have seen in the [Intro to PyTorch](https://github.com/priyammaz/HAL-DL-From-Scratch/tree/main/Intro%20to%20PyTorch), a Deep Learning model is really just a structured bag of parameters. In the Linear Regression example, we had only two parameters, a weight $W_1$ and a Bias $W_0$. The only models we can create by multiplying and adding numbers together is a linear model. The Dense MNIST model we then made after was a deeper model with more parameters and connections, but we were still only multiplying and adding numbers together, therefore we can only represent a linear relationship. To fix this we then added in some ReLU (Rectified Linear Unit) activation functions that would allow us to introduce non-linearity to our model! All these weights that are randomly initialized when we define the model are then optimized and tuned so we have the least error between our predictions and true value. 

If we took our model and saved the parameters that define it, this would be known as a **pre-trained** model. Transfer learning is then the idea that, if we already have a model that can predict handwritten digits very well (0 through 9), and we have a new dataset of the lowercase alphabet (a through z), do we really need to train a new model from scratch? Or can we just grab our pre-trained model and its already optimized weights and finetune it to the new dataset?

Intuitively you can think about it this way. Lets say we show a kid some pictures of dogs and cats (like the dataset we are working on). After we show them some images of these dogs and cats, they will be able to classify them pretty well all on their own! Now we want to give them a new task, classify between tigers and wolves. Obviously tigers are not the same as cats and wolves are not the same as dogs, but they do share a bunch of similarities right? So the kid would then use their previous knowledge (The **pre-trained** model) and exapand it to be able to do this new task.

## AlexNet and Convolutions
We will be skipping ahead a bit here. In the next lesson, PyTorch for Vision, we will be covering in detail the ideas for convolutions and why they are much better that Dense Linear layers for Images. For now, pretend the model we will use is some black box that takes in an image, and outputs some probabilities of belonging to different classes. The specific model we will look at is AlexNet, which was probably the first real evidence of the power of deep learning in 2012. It was trained on the ImageNet task (predict across 1000 classes of images) and beat all other methods by a significant margin. We will be loading a PyTorch defined version of this model, but don't worry, we will implement this entire model from scratch in the next lesson! I am just trying to offer some intuition for Deep Learning before we get into the weeds!


## More Intution about Deep Learning 

As we have seen, the reason it is called "Deep" Learning is because the model physically has depth and many layers of computation. Because of this we actually get some interesting properties!

![Image](https://raw.githubusercontent.com/priyammaz/HAL-DL-From-Scratch/main/src/visuals/conv_feature_extracts.jpeg)

[credit](https://anhvnn.wordpress.com/2018/02/01/deep-learning-computer-vision-and-convolutional-neural-networks/)

This is a common image you will see when exploring Deep Learning. You can see that earlier features that are learned are focused on simple things to find in an image, like edges, lines, etc... Regardless of the Image problem at hand, we always have to detect these features, so the underlying weights in a pretrained model that encode this can easily be used. As we move forward through the model, the extracted features in the image become much more specific to the dataset, and in this case there is some type of face detection. It is really cool though that convolutions (Again a black box that does some type of image processing) can extract abstract features at such a high level!

Now lets say we want to use this pretrained model and now detect Dogs instead. Well on its own, this model would do poorly as it is optimized to find human faces! But also the intial layers that do low level image feautures are probably fine to leave alone. Therefore, the typical strategy is to keep the earlier parts of the model static (don't allow gradient updates) and then finetune the later layers.

There are some benefits for this:
- We can pretrain a model on a giant dataset, and then finetune it to a similar problem that we have limited data for
- We can control and avoid problems like overfitting better

In [1]:
### Imports ###
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import AlexNet, AlexNet_Weights
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from tqdm import tqdm
import numpy as np

import warnings
warnings.filterwarnings("ignore")

### Recap From Before

We will be skipping the details of the setup for the DataLoader and everything here. If you have any confusion, go back to my [PyTorch DataLoader](https://github.com/priyammaz/HAL-DL-From-Scratch/tree/main/PyTorch%20DataLoaders) tutorial where I go in depth about how this works!

In [3]:
### Build Cats vs Dogs Dataset ###
PATH_TO_DATA = "../../data/PetImages/"

### DEFINE TRANSFORMATIONS ###
normalizer = transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]) ### IMAGENET MEAN/STD ###
train_transforms = transforms.Compose([
                                        transforms.Resize((224,224)),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        normalizer
                                      ])


dataset = ImageFolder(PATH_TO_DATA, transform=train_transforms)

train_samples, test_samples = int(0.9 * len(dataset)), len(dataset) - int(0.9 * len(dataset))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, lengths=[train_samples, test_samples])

### Lets Now Load the Model!!
We will now load the AlexNet model from PyTorch and then poke around a bit to see how these models are typically defined! Below you will see a ton of thigns you haven't seen before but its ok! We will go into detail again in the next tutorial. For now lets just take a rought look at whats going on:

In [4]:
model = AlexNet()
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

All PyTorch models are saved like this, where every layer has a name and we can access them individually. For example, if we want to look at the 1st linear layer in the classifier we can do it this way:

```
model.classifier[1]
```

In [5]:
model.classifier[1]

Linear(in_features=9216, out_features=4096, bias=True)

### What is the Output of the Model?

Notice the very last linear layer looks like this:

```
Linear(in_features=4096, out_features=1000, bias=True)
```

We can clearly see here that the model is outputing 1000 values because the ImageNet task requires us to predict across 1000 classes. As we start thinking about how we want to use this model, if we have a binary classification problem (Cats vs Dogs), then we will need to change this last linear layer so its output is only 1 value (like we had in logistic regression. We will then apply a softmax where if we get a value greater than 50%

**Lets Update the Model!**
We know that we can access the last layer of the model by doing *model.classifier[6]* and we want to change this to a linear layer. We saw in the [Intro to PyTorch](https://github.com/priyammaz/HAL-DL-From-Scratch/tree/main/Intro%20to%20PyTorch) that we can define a linear layer as:

```
Linear(in_features, out_features)
```

but we have to make sure that the input features of the new layer matches the output features of the previous Linear layer. We can see in the 4th Linear layer that the output is 4096 so we will ensure to keep that as our input!

In [6]:
model = AlexNet()
model.classifier[6] = nn.Linear(4096, 2)
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

Now we can clearly see that our model is outputting 2 features rather than 1000! This then gives us everything we need, so lets pass a test tesor through it to make sure its all functional, and then we can do a bit more exploration. As we said before, the common shape for Image data is:

```
[Batch Size x Channels x Image Height x Image Width]
```

We will just make a dummy tensor that matches the CatsVsDogs dataset we made that will have the shape [16 x 3 224 x 224]

In [7]:
rand_data = torch.rand(16,3,224,224)
model_output = model(rand_data)
model_output.shape

torch.Size([16, 2])

We can see that we passed in 16 images and the output is 2 classes per image, so the model is fully functional!!

### Checking out the Model Parameters 
The attribute **.named_parameters()** that a model has allows us to iterate through all the names and parameters of the model! You will notice that the parameters are also stored as Tensors and these are the number that are updated through optimization! We can add up the number of parameters in the model and see that after we reduced the final output dimension to 2, we have roughly 57 Million parameters! That may sound like a lot but most Deep Learning models today have Billions of parameters.

In [8]:
total_parameters = 0
for name, params in model.named_parameters():
    num_params = int(torch.prod(torch.tensor(params.shape)))
    print(name,":", params.shape, "Num Parameters:", num_params)
    total_parameters += num_params
    
print("------------------------")
print("Total Parameters in Model", total_parameters)
    

features.0.weight : torch.Size([64, 3, 11, 11]) Num Parameters: 23232
features.0.bias : torch.Size([64]) Num Parameters: 64
features.3.weight : torch.Size([192, 64, 5, 5]) Num Parameters: 307200
features.3.bias : torch.Size([192]) Num Parameters: 192
features.6.weight : torch.Size([384, 192, 3, 3]) Num Parameters: 663552
features.6.bias : torch.Size([384]) Num Parameters: 384
features.8.weight : torch.Size([256, 384, 3, 3]) Num Parameters: 884736
features.8.bias : torch.Size([256]) Num Parameters: 256
features.10.weight : torch.Size([256, 256, 3, 3]) Num Parameters: 589824
features.10.bias : torch.Size([256]) Num Parameters: 256
classifier.1.weight : torch.Size([4096, 9216]) Num Parameters: 37748736
classifier.1.bias : torch.Size([4096]) Num Parameters: 4096
classifier.4.weight : torch.Size([4096, 4096]) Num Parameters: 16777216
classifier.4.bias : torch.Size([4096]) Num Parameters: 4096
classifier.6.weight : torch.Size([2, 4096]) Num Parameters: 8192
classifier.6.bias : torch.Size([2]

### Lets Train This Model From Scratch!
Our model is currently randomly initialized, so we will be training the model from scratch just to see how it does!

In [9]:
### SELECT DEVICE ###
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on Device {DEVICE}")

### LOAD IN and Modify AlexNet Model ###
model = AlexNet()
model.classifier[6] = nn.Linear(4096, 2)
model = model.to(DEVICE)

### MODEL TRAINING INPUTS ###
epochs = 5
optimizer = optim.Adam(params=model.parameters(), lr=0.0001)
loss_fn = nn.CrossEntropyLoss()
batch_size = 128

### BUILD DATALOADERS ###
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

def train(model, device, epochs, optimizer, loss_fn, batch_size, trainloader, valloader):
    log_training = {"epoch": [],
                    "training_loss": [],
                    "training_acc": [],
                    "validation_loss": [],
                    "validation_acc": []}

    for epoch in range(1, epochs + 1):
        print(f"Starting Epoch {epoch}")
        training_losses, training_accuracies = [], []
        validation_losses, validation_accuracies = [], []

        for image, label in tqdm(trainloader):
            image, label = image.to(DEVICE), label.to(DEVICE)
            optimizer.zero_grad()
            out = model.forward(image)
        
            ### CALCULATE LOSS ##
            loss = loss_fn(out, label)
            training_losses.append(loss.item())

            ### CALCULATE ACCURACY ###
            predictions = torch.argmax(out, axis=1)
            accuracy = (predictions == label).sum() / len(predictions)
            training_accuracies.append(accuracy.item())

            loss.backward()
            optimizer.step()

        for image, label in tqdm(valloader):
            image, label = image.to(DEVICE), label.to(DEVICE)
            with torch.no_grad():
                out = model.forward(image)

                ### CALCULATE LOSS ##
                loss = loss_fn(out, label)
                validation_losses.append(loss.item())

                ### CALCULATE ACCURACY ###
                predictions = torch.argmax(out, axis=1)
                accuracy = (predictions == label).sum() / len(predictions)
                validation_accuracies.append(accuracy.item())

        training_loss_mean, training_acc_mean = np.mean(training_losses), np.mean(training_accuracies)
        valid_loss_mean, valid_acc_mean = np.mean(validation_losses), np.mean(validation_accuracies)

        log_training["epoch"].append(epoch)
        log_training["training_loss"].append(training_loss_mean)
        log_training["training_acc"].append(training_acc_mean)
        log_training["validation_loss"].append(valid_loss_mean)
        log_training["validation_acc"].append(valid_acc_mean)

        print("Training Loss:", training_loss_mean) 
        print("Training Acc:", training_acc_mean)
        print("Validation Loss:", valid_loss_mean)
        print("Validation Acc:", valid_acc_mean)
        
    return log_training, model


random_init_logs, model = train(model=model,
                                device=DEVICE,
                                epochs=epochs,
                                optimizer=optimizer,
                                loss_fn=loss_fn,
                                batch_size=batch_size,
                                trainloader=trainloader,
                                valloader=valloader)



Training on Device cuda
Starting Epoch 1


100%|██████████| 20/20 [00:03<00:00,  6.09it/s]


Training Loss: 0.6141784023493528
Training Acc: 0.6454298316755078
Validation Loss: 0.5807640105485916
Validation Acc: 0.7105594754219056
Starting Epoch 2


100%|██████████| 20/20 [00:03<00:00,  6.22it/s]


Training Loss: 0.4646523620255969
Training Acc: 0.7787306129255078
Validation Loss: 0.41430944204330444
Validation Acc: 0.8121723771095276
Starting Epoch 3


...

Training Loss: 0.37520143254236743
Training Acc: 0.8337768255309626
Validation Loss: 0.37039587572216986
Validation Acc: 0.8431829631328582
Starting Epoch 4


100%|██████████| 20/20 [00:03<00:00,  6.26it/s]


Training Loss: 0.3267671912908554
Training Acc: 0.8572311215102673
Validation Loss: 0.3356307603418827
Validation Acc: 0.8529233872890473
Starting Epoch 5


100%|██████████| 20/20 [00:03<00:00,  6.05it/s]

Training Loss: 0.27911953297867015
Training Acc: 0.8793369453739036
Validation Loss: 0.4088128298521042
Validation Acc: 0.8215977817773819


## Lets Now Load our PreTrained Weights
We will still be training the entire model end to end, but it will now start with the pretrained weights from ImageNet.

**Note**: The pretrained model by default has a final linear layer that outputs to 1000 classes! When we swap this last portion out with a new Linear layer that outputs to 2 classes, only that layer will be randomly initialized. The remaining calculations (previous linear layers, convolutions, etc...) are all still using the pretrained valued.

In [10]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model.classifier[6] = nn.Linear(4096, 2)
model = model.to(DEVICE)

### MODEL TRAINING INPUTS ###
epochs = 2
optimizer = optim.Adam(params=model.parameters(), lr=0.0001)
loss_fn = nn.CrossEntropyLoss()
batch_size = 128

### BUILD DATALOADERS ###
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


random_init_logs, model = train(model=model,
                                device=DEVICE,
                                epochs=epochs,
                                optimizer=optimizer,
                                loss_fn=loss_fn,
                                batch_size=batch_size,
                                trainloader=trainloader,
                                valloader=valloader)





Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /u/priyamm2/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /u/priyamm2/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:00<00:00, 360MB/s] 


Starting Epoch 1


100%|██████████| 20/20 [00:03<00:00,  6.14it/s]


Training Loss: 0.11720726574474777
Training Acc: 0.9521040482954546
Validation Loss: 0.08458526078611613
Validation Acc: 0.9675025194883347
Starting Epoch 2


100%|██████████| 20/20 [00:03<00:00,  6.12it/s]

Training Loss: 0.06550350044430657
Training Acc: 0.9753196022727273
Validation Loss: 0.08191037848591805
Validation Acc: 0.9663810461759568


Notice in a less number of epochs, we have beat the preformance of the "From Scratch" model greatly. Obviously if we kept training the randomized model more, we would have reached a similar performance but I am trying to show the benefits if you had lower resources for compute. 

## Load PreTrained Weights but Only Train the Final Classifier Layer
Like we had mentioned previously, the entire model starts with the pretrained weights, but when we swap the last linear layer with one that outputs to 2 classes, that layer will become randomly initialized. A common technique is then to freeze the remaining layers and only train this classifier head. To do this lets look at a flag that many tensors have in PyTorch!

**Note**: All Tensors (atleast by default) in your PyTorch model will have this. You will see that I am only showing the first layer that includes "bias" in the name. This is only for visualization as the bias tensors are pretty small compared to the massive weight tensors that belong to the convolutions. The idea is the same regardless though as the bias is also a learnable parameter!

In [11]:
for name, param in model.named_parameters():
    if "bias" in name:
        print(name)
        print(param)
        break

features.0.bias
Parameter containing:
tensor([-0.9703, -2.8073, -0.0375, -0.0788, -0.1161,  0.0238, -0.0757, -1.4175,
         1.6432, -0.1000, -0.0171, -0.1288, -0.0668, -0.0342, -0.0738, -1.2972,
        -0.0514,  0.0112, -0.1025, -1.1888, -0.1388, -0.0491, -0.0789, -0.0414,
        -0.0983, -0.0693, -1.9378, -0.0864, -0.1402, -0.1977, -0.1296, -2.0097,
        -0.0498, -0.0618, -0.0372, -0.3867, -2.7825,  0.6598, -0.1656, -2.1295,
         0.0520, -0.0281, -0.1701, -0.0598, -0.4210, -1.9407, -1.2088,  0.0135,
        -0.1087, -0.0248, -0.1512, -1.8507, -0.0932, -0.0180, -0.0695, -0.0579,
        -0.0633, -0.0725, -1.2674, -0.1174, -0.0442, -0.3267,  0.0498, -0.0144],
       device='cuda:0', requires_grad=True)


Notice the flag at the end of the Tensor: 
```
required_grad = True
```

This indicates that when PyTorch is optimizing the model, this tensor allows for gradient updates. Therefore, if we want to turn this off for this layer, we just need to turn that flag to False. We will want to repeat this process of turning off gradient updates for all layers except for our last one!

In [12]:
for name, param in model.named_parameters():
    print(name)

features.0.weight
features.0.bias
features.3.weight
features.3.bias
features.6.weight
features.6.bias
features.8.weight
features.8.bias
features.10.weight
features.10.bias
classifier.1.weight
classifier.1.bias
classifier.4.weight
classifier.4.bias
classifier.6.weight
classifier.6.bias


As we can see, the name of our last classifier includes "classifier.6", so we will use that to turn the gradients off on everything else!

In [13]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model.classifier[6] = nn.Linear(4096, 2)

# Check the name of all the parameters
for name, param in model.named_parameters():
    if "classifier.6" not in name:
        param.requires_grad_(False) # Inplace turn of gradient updates

        
for name, param in model.named_parameters():
    if "bias" in name:
        print(name)
        print(param)
        break

Using cache found in /u/priyamm2/.cache/torch/hub/pytorch_vision_v0.10.0


features.0.bias
Parameter containing:
tensor([-0.9705, -2.8070, -0.0371, -0.0795, -0.1159,  0.0252, -0.0752, -1.4181,
         1.6454, -0.0990, -0.0161, -0.1282, -0.0658, -0.0345, -0.0743, -1.2977,
        -0.0505,  0.0121, -0.1013, -1.1887, -0.1380, -0.0492, -0.0789, -0.0405,
        -0.0958, -0.0705, -1.9374, -0.0850, -0.1388, -0.1968, -0.1279, -2.0095,
        -0.0476, -0.0604, -0.0351, -0.3843, -2.7823,  0.6605, -0.1655, -2.1293,
         0.0543, -0.0274, -0.1703, -0.0593, -0.4215, -1.9394, -1.2094,  0.0153,
        -0.1081, -0.0248, -0.1503, -1.8516, -0.0928, -0.0177, -0.0700, -0.0582,
        -0.0630, -0.0721, -1.2678, -0.1176, -0.0441, -0.3259,  0.0507, -0.0146])


Notice now that the Requires Grad flag is now gone! We should be good to go now to retrain the model one more time. 

In [14]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model.classifier[6] = nn.Linear(4096, 2)

# Check the name of all the parameters
for name, param in model.named_parameters():
    if "classifier.6" not in name:
        param.requires_grad_(False) # Inplace turn of gradient updates

model = model.to(DEVICE)

### MODEL TRAINING INPUTS ###
epochs = 2
optimizer = optim.Adam(params=model.parameters(), lr=0.0001)
loss_fn = nn.CrossEntropyLoss()
batch_size = 128

### BUILD DATALOADERS ###
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


random_init_logs, model = train(model=model,
                                device=DEVICE,
                                epochs=epochs,
                                optimizer=optimizer,
                                loss_fn=loss_fn,
                                batch_size=batch_size,
                                trainloader=trainloader,
                                valloader=valloader)



Using cache found in /u/priyamm2/.cache/torch/hub/pytorch_vision_v0.10.0


Starting Epoch 1


100%|██████████| 20/20 [00:03<00:00,  6.21it/s]


Training Loss: 0.2096461996182122
Training Acc: 0.9151998236775398
Validation Loss: 0.12757282555103303
Validation Acc: 0.9522933453321457
Starting Epoch 2


100%|██████████| 20/20 [00:03<00:00,  6.17it/s]

Training Loss: 0.12724176078865473
Training Acc: 0.9502433010122993
Validation Loss: 0.11188035123050213
Validation Acc: 0.9549773156642913


## Lets Roundup All the Ideas Now!
Pretrained models allow us to "Transfer" knowledge from a previous task to a new one. The main benefits to this are as follows:
- Less data needed to train model and allows us to work on niche tasks even with these expressive datasets
- Less concern for Overfitting. Typically, if you train a large model on a small dataset, it will quickly memorize the data. Instead, we can train just a small part of the model and keep the rest as a feature extractor. 
- Less compute! Deep Learning takes long enough already to train. Atleast we have less parameters to do gradient updating on, so it saves us quite a bit of compute. 


Pretrained models are the current future for deep learning. It has become the gold standard to take large language, vision, speech, and any other modality and transfer it to new tasks. Even more interesting is, the models have become so powerful, they can often transfer to completely unrelated tasks (Using NLP based models for Protein Analysis). This technique also greatly democratizes Deep Learning to many because if the models are easier to train and dont require entire data centers like the Pretraining stage did, then we can all enjoy solving the problems we are interested in!